<a href="https://colab.research.google.com/github/edgekrtl/100-days-of-code/blob/master/IBB_Traffic_Data_Visualisation_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


In [4]:

# Step 1: Data Acquisition

# Load Hourly Traffic Density Dataset
traffic_density_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/traffic_density_202304.csv')
print(traffic_density_data.head)



<bound method NDFrame.head of                    DATE_TIME  LONGITUDE   LATITUDE GEOHASH  MINIMUM_SPEED  \
0        2023-04-01 00:00:00  29.360962  40.816956  sxkbhn            128   
1        2023-04-01 00:00:00  28.855591  41.047668  sxk988             48   
2        2023-04-01 00:00:00  28.185425  41.075134  sxk19m            110   
3        2023-04-01 00:00:00  28.789673  41.069641  sxk3xh             85   
4        2023-04-01 00:00:00  29.141235  40.899353  sxk8xj             91   
...                      ...        ...        ...     ...            ...   
1611916  2023-04-30 23:00:00  29.273071  40.965271  sxkc41            162   
1611917  2023-04-30 23:00:00  28.899536  41.058655  sxk99d             72   
1611918  2023-04-30 23:00:00  29.119263  40.981750  sxk9ns             45   
1611919  2023-04-30 23:00:00  29.097290  41.003723  sxk9q0            130   
1611920  2023-04-30 23:00:00  29.108276  41.047668  sxk9w2             49   

         MAXIMUM_SPEED  AVERAGE_SPEED  NUMBER

In [5]:
# Load Accident Announcements Dataset
accident_announcements_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/traffic_announcement.csv')
print(accident_announcements_data.head)

<bound method NDFrame.head of         ANNOUNCEMENT_ID       ANNOUNCEMENT_STARTING_DATETIME  \
0                 32396   2013-02-12 14:29:39.94 +0000 +0000   
1                 32764   2013-02-18 08:48:15.21 +0000 +0000   
2                 33201   2013-02-25 10:33:39.77 +0000 +0000   
3                 33208   2013-02-25 11:43:14.27 +0000 +0000   
4                 33211   2013-02-25 12:07:50.58 +0000 +0000   
...                 ...                                  ...   
160662           920181  2022-08-02 18:19:49.967 +0000 +0000   
160663           920182  2022-08-02 18:31:28.133 +0000 +0000   
160664           920184  2022-08-02 18:37:26.087 +0000 +0000   
160665           920185  2022-08-02 18:38:29.123 +0000 +0000   
160666           920186   2022-08-02 18:42:33.57 +0000 +0000   

              ANNOUNCEMENT_ENDING_DATETIME  \
0          2013-02-12 15:12:43 +0000 +0000   
1          2013-02-18 10:37:24 +0000 +0000   
2          2013-02-25 13:18:59 +0000 +0000   
3       2013-02-2

<ipython-input-5-6b072375f0a8>:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_announcements_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/traffic_announcement.csv')


In [6]:
# Step 3: Data Visualization

# Create a map of Istanbul
fig = go.Figure()


In [7]:

# Add the hourly traffic density information to the map
fig.add_trace(go.Scattermapbox(
    lat=traffic_density_data['LATITUDE'],
    lon=traffic_density_data['LONGITUDE'],
    mode='markers',
    marker=dict(
        size=traffic_density_data['NUMBER_OF_VEHICLES'],
        color=traffic_density_data['AVERAGE_SPEED'],
        colorscale='Viridis',
        opacity=0.7
    ),
    hovertext='Number of Vehicles: ' + traffic_density_data['NUMBER_OF_VEHICLES'].astype(str) +
               '<br>Average Speed: ' + traffic_density_data['AVERAGE_SPEED'].astype(str),
    hoverinfo='text'
))


In [24]:

# Add the accident announcements to the map
fig.add_trace(go.Scattermapbox(
    lat=accident_announcements_data['LATITUDE'],
    lon=accident_announcements_data['LONGITUDE'],
    mode='markers',
    marker=dict(
        color='red',
        size=10,
        symbol='circle'
    ),
    hovertext='Accident Announcement',
    hoverinfo='text'
))


In [ ]:

# Update the layout of the map
fig.update_layout(
    mapbox=dict(
        style='carto-positron',
        center=dict(lat=41.0082, lon=28.9784),
        zoom=10
    ),
    showlegend=False
)


In [11]:

# Step 4: Timelapse Animation

# Create an animation frame for each hour
frames = []
for hour in range(24):
    # Filter the data for the current hour
    hour_data = traffic_density_data[traffic_density_data['DATE_TIME'] == hour]

    # Create a frame for the current hour
    frame = go.Frame(
        data=[go.Scattermapbox(
            lat=hour_data['LATITUDE'],
            lon=hour_data['LONGITUDE'],
            mode='markers',
            marker=dict(
                size=hour_data['NUMBER_OF_VEHICLES'],
                color=hour_data['AVERAGE_SPEED'],
                colorscale='Viridis',
                opacity=0.7
            ),
            hovertext='Number of Vehicles: ' + hour_data['NUMBER_OF_VEHICLES'].astype(str) +
                       '<br>Average Speed: ' + hour_data['AVERAGE_SPEED'].astype(str),
            hoverinfo='text'
        )],
        name=f'Hour {hour}'
    )

    frames.append(frame)

# Add the frames to the map animation
fig.frames = frames


In [12]:
# Step 5: Analysis and Insights

# Perform analysis and extract insights from the data
# Step 5: Analysis and Insights

# Traffic Patterns Analysis
peak_traffic_hours = traffic_density_data.groupby('DATE_TIME')['NUMBER_OF_VEHICLES'].sum().sort_values(ascending=False).head(5)
print("Peak Traffic Hours:")
print(peak_traffic_hours)

# Congestion Hotspots Analysis
congestion_locations = traffic_density_data.groupby(['LATITUDE', 'LONGITUDE'])['NUMBER_OF_VEHICLES'].mean().sort_values(ascending=False).head(5)
print("Congestion Hotspots:")
print(congestion_locations)

# Accident Analysis
accident_counts = accident_announcements_data.groupby(['LATITUDE', 'LONGITUDE']).size().sort_values(ascending=False).head(5)
print("Stoppings Counts by Location:")
print(accident_counts)

# Safety Measures
common_accident_types = accident_announcements_data['ANNOUNCEMENT_TYPE_DESC'].value_counts().head(5)
print("Most Common Stopping Types:")
print(common_accident_types)

# Traffic Optimization
low_speed_areas = traffic_density_data[traffic_density_data['AVERAGE_SPEED'] < 30]
low_speed_areas_count = low_speed_areas.groupby(['LATITUDE', 'LONGITUDE']).size().sort_values(ascending=False).head(5)
print("Areas with Low Speeds:")
print(low_speed_areas_count)




Peak Traffic Hours:
DATE_TIME
2023-04-08 13:00:00    357233
2023-04-08 12:00:00    349948
2023-04-08 16:00:00    341757
2023-04-08 15:00:00    341061
2023-04-08 14:00:00    340434
Name: NUMBER_OF_VEHICLES, dtype: int64
Congestion Hotspots:
LATITUDE   LONGITUDE
41.064148  28.811646    468.143066
41.058655  28.811646    444.741606
40.992737  29.152222    435.845255
40.937805  29.328003    435.719708
41.064148  28.833618    412.528467
Name: NUMBER_OF_VEHICLES, dtype: float64
Stoppings Counts by Location:
LATITUDE     LONGITUDE  
29.04711500  41.09048400    997
28.85158700  40.99108600    716
28.76879300  40.98605300    653
28.94932000  41.05160000    573
41.00563861  28.67400636    460
dtype: int64
Most Common Stopping Types:
Kaza Bildirimi            77916
Araç Arızası              54947
Bakım-Onarım Çalışması    17463
Yoğun Trafik               5811
Çevre Düzenlemesi          1328
Name: ANNOUNCEMENT_TYPE_DESC, dtype: int64
Areas with Low Speeds:
LATITUDE   LONGITUDE
41.053162  28.800659

In [13]:

# Predictive Analysis (Example: Using Linear Regression)
from sklearn.linear_model import LinearRegression

# Prepare the data for predictive analysis
traffic_density_data['DATE_TIME'] = pd.to_datetime(traffic_density_data['DATE_TIME'])
traffic_density_data['day_of_week'] = traffic_density_data['DATE_TIME'].dt.dayofweek
traffic_density_data['hour_of_day'] = traffic_density_data['DATE_TIME'].dt.hour

# Train a linear regression model to predict traffic density
X = traffic_density_data[['day_of_week', 'hour_of_day']]
y = traffic_density_data['NUMBER_OF_VEHICLES']

model = LinearRegression()
model.fit(X, y)

# Predict traffic density for a specific day and hour
day = 10  # Example: 10th day of the month
hour = 8  # Example: 8 AM
predicted_density = model.predict([[day, hour]])
print(f"Predicted Traffic Density on the 10th day at 8 AM: {predicted_density}")



Predicted Traffic Density on the 10th day at 8 AM: [51.58209245]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



In [14]:

# Update the animation settings
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}])
            ],
            showactive=False,
            direction='left',
            x=0.1,
            y=0,
            xanchor='right',
            yanchor='top'
        )
    ]
)




In [23]:
import plotly.io as pio

# Render the map visualization
pio.renderers.default = "iframe"  # Choose the appropriate renderer ("notebook" or "iframe")

# Save the map visualization as HTML
fig.write_html('map_visualization.html')
